# Overview
The **objective** of this notebook is to present the flow chart of conducting data assimilation on [PFLOTRAN](https://www.pflotran.org/) by using [DART](https://www.image.ucar.edu/DAReS/DART/). Briefly, the procedures are as follows:
- [ ] [Install PFLOTRAN and DART](#installation)
- [x] [Configuration](#parameter): define directories, file locations, and other parameters
- [x] [PFLOTRAN preparation](#pflotran_prepare): generate PFLOTRAN input files
- [x] [PFLOTRAN model spin-up](#pflotran_spinup): conduct model spin-up
- [x] [DART files preparation](#dart_prepare): add new DART quantities, prepare DART input namelists, prepare DART prior data, prepare observations in DART format, and check ```model_mod``` interface
- [ ] [Run DART and PFLOTRAN](#run_dart_pflotran): run the shell script for integrating DART filter and PFLOTRAN model

Here, we perform inverse modeling on a 1D thermal model for illustration. The model assimilates temperature observation to update its parameters (i.e., flow flux, porosity, and thermal conductivity). For now, the ensemble Kalman filter (EnKF) is used for assimilation.

<a id='installation'></a>
# PFLOTRAN and DART Installation

<a id='parameter'></a>
# Configuration

In [1]:
import re
import pickle

In [7]:
# Directories
obs_kind_dir    = '../obs_kind/'        # Directory for defining default DART generic quantity  
obs_type_dir    = '../obs_type/'        # Directory for mapping observation variable with DART generic quantity
utils_dir       = '../utils/'           # Directory for utility files
work_dir        = '../work/'            # Directory for compiling and running DART files
pflotran_in_dir = '../pflotran_input/'  # Directory for saving PFLOTRAN input files
pflotran_out_dir= '../pflotran_output/' # Directory for saving PFLOTRAN output files
dart_data_dir   = '../dart_inout/'      # Directory for saving DART in-out files

# DART file names
def_obs_kind   = obs_kind_dir+'DEFAULT_obs_kind_mod.f90'   # The default DART generic quantity file
obs_type       = obs_type_dir+'obs_def_pflotran_mod.f90'   # The map between DART generic quantities and observation variables
input_nml      = work_dir+'input.nml'                      # The input namelists used by DART programs
input_nml_dict = work_dir+'inputnml.p'                     # The pickle file for saving input namelists in dictionary format

# PFLOTRAN file names
pflotran_sh   = utils_dir+'pflotran.sh'                                 # Script for running PFLOTRAN
pflotran_exe  = '/Users/jian449/Codes/pflotran/src/pflotran/pflotran'   # Location of the executable PFLOTRAN file
pflotran_in   = pflotran_in_dir+'pflotran.in'                           # PFLOTRAN input deck file
pflotran_para = pflotran_in_dir+'parameter.h5'                          # PFLOTRAN parameter HDF file
pflotran_out  = pflotran_out_dir+'R[ENS].h5'                            # PFLOTRAN output HDF filename template for each ensemble [ENS]

# Data file names, including observations and model input/output
obs_original = pflotran_in_dir+'temperature.csv'          # The original observation file in CSV format
obs_nc       = pflotran_in_dir+'obs_pflotran.nc'          # The converted observation file in NetCDF format
obs_dart     = dart_data_dir+'obs_seq_pflotran.out'       # The converted observation file in DART format
dart_prior_nc= dart_data_dir+'prior_R[ENS].nc'            # The NetCDF filename template for DART's prior data of each ensemble [ENS]
dart_prior_template = re.sub(r"\[ENS\]",'template',dart_prior_nc)  # The NetCDF filename for DART
dart_input_list = dart_data_dir+"filter_input_list.txt"   # The list of DART prior files for all ensembles
dart_output_list = dart_data_dir+"filter_output_list.txt" # The list of DART posterior files for all ensembles

# Some shell scripts or executable files
convert_nc              = work_dir+'convert_nc'           # The executable file to convert observation from NetCDF to DART formats
compile_convert_nc      = work_dir+'dart_seq_convert.csh' # Shell script for generating the executable file to convert observation from NetCDF to DART formats
compile_model_check_mod = work_dir+'check_model_mod.csh'  # Shell script for checking model_mod.F90 file
run_filter              = work_dir+'run_filter.csh'       # Shell script for running DART filter and PFLOTRAN
advance_model           = work_dir+'advance_model.csh'    # Shell script for forward simulation of PFLOTRAN

# Utility file names
csv_to_nc          = utils_dir+'csv2nc.py'                  # Python script for converting raw observation to NetCDF file
to_dartqty         = utils_dir+'list2dartqty.py'            # Python script for reading a list of variable names and adding new DART quantities      
prep_pflotran_in   = utils_dir+'prepare_pflotran_inpara.py' # Python script for preparing PFLOTRAN.in and parameter.h5 files
prep_convert_nc    = utils_dir+'prepare_convert_nc.py'      # Python script for preparing convert_nc.F90 script
prep_prior_nc      = utils_dir+'prepare_prior_nc.py'        # Python script for preparing the prior NetCDF files for DART
prep_inputnml      = utils_dir+'prepare_input_nml.py'       # Python script for preparing the input.nml file

In [8]:
# MPI settings
mpi_exe = '/usr/local/bin/mpirun'  # The location of mpirun
ncore   = 1                        # The number of MPI cores used

In [9]:
# Data assimilation configurations
# More need to be added...
# And later on, these DA setting can be saved in a txt or pickel file for further loading
# obs_timestep  = 300.0  # second
obs_resolution  = 300.0  # second
obs_error     = 0.1    # observation error
spinup        = 1      # whether model spinup
spinup_length = 0.5    # spinup time (day)
nens          = 30     # number of ensembles

# Assimilation time window time_step_days+time_step_seconds
time_step_days    = 0   # assimilation time window/step (day)
time_step_seconds = 600 # assimilation time window/step  (second)=

# Specify PFLOTRAN variables used as observation and state vector/parameters in DART
obs_var_set = ['TEMPERATURE']
para_set    = ['FLOW_FLUX','POROSITY','THERMAL_CONDUCTIVITY']
# state_set   = ['LIQUID_SATURATION','LIQUID_PRESSURE', 'TEST_VARIABLE', 'TEST_VARIABLEAAA'
pflotran_parastate_set = obs_var_set + para_set

In [10]:
# Save the above parameters in pickle???

<a id='pflotran_prepare'></a>
# PFLOTRAN preparation
*Here, we use Kewei's 1D thermal model as an example for generating PFLOTRAN input card and parameter.h5.*

In this section, the following procedures are performed:
- generate PFLOTRAN input deck file ```PFLOTRAN.in```
- generate the parameter files in HDF 5, ```parameter.h5```, used by PFLOTRAN input deck file

## Generate ```PFLOTRAN.in``` and ```parameter.h5```

```PFLOTRAN.in``` should be prepared by users.

- Run: ```prepare_pflotran_inpara.py```
- Code input arguments:
    - <span style="background-color:yellow">pflotran_in</span>: filename for ```pflotran.in```
    - <span style="background-color:yellow">pflotran_para</span>: filename for ```parameter.h5```
    - <span style="background-color:yellow">obs_resolution, obs_error, nens, spinup, spinup_length</span>: data assimilation settings (i.e., observation timestep, observation error, number of ensemble, whether it is spinup, **to be revised**)

In [11]:
%%script bash -s "$prep_pflotran_in" "$pflotran_in" "$pflotran_para" "$obs_resolution" "$obs_error" "$nens" "$spinup" "$spinup_length"
python $1 $2 $3 $4 $5 $6 $7 $8

Finished generating the input card for PFLOTRAN...
Finished generating the DBASE for PFLOTRAN...


In [12]:
%%script bash -s "$pflotran_in"
head $1

#Description: 1D thermal

SIMULATION
  SIMULATION_TYPE SUBSURFACE
  PROCESS_MODELS
    SUBSURFACE_FLOW FLOW
      MODE TH
#      OPTIONS
#	REVERT_PARAMETERS_ON_RESTART
#      /


<a id='pflotran_spinup'></a>
# PFLOTRAN model spin-up
Take in the ```pflotran.in``` and ```parameter.h5``` files and conduct the model spin-up by running ```pflotran.sh``` file. The ```pflotran.sh``` is a simple shell script executing ensemble simulation of PFLOTRAN by using MPI.

**Run the code**
- Run: ```prepare_pflotran_inpara.py```
- Code input arguments:
    - <span style="background-color:yellow">pflotran_exe</span>: location of the executable PFLOTRAN
    - <span style="background-color:yellow">pflotran_in</span>: filename for ```pflotran.in```
    - <span style="background-color:yellow">pflotran_out_dir</span>: directory of PFLOTRAN output
    - <span style="background-color:yellow">nens</span>: number of ensemble
    - <span style="background-color:yellow">mpi_exe, ncore</span>: location of mpirun and number of cpu cores

In [13]:
%%script bash -s "$pflotran_sh" "$pflotran_exe" "$pflotran_in" "$pflotran_out_dir" "$nens" "$mpi_exe" "$ncore"
echo $3
$1 $2 $3 $4 $5 $6 $7

../pflotran_input/pflotran.in
 here
          30
------------------------------ Provenance --------------------------------------
pflotran_compile_date_time = unknown
pflotran_compile_user = unknown
pflotran_compile_hostname = unknown
pflotran_changeset = unknown
pflotran_status = unknown
petsc_changeset = unknown
petsc_status = unknown
--------------------------------------------------------------------------------
 "grid_structured_type" set to default value.
 Opening hdf5 file: ../pflotran_input/parameter.h5
 pflotran card:: TIMESTEPPER
 pflotran card:: TIMESTEPPER
 pflotran card:: NEWTON_SOLVER
 pflotran card:: LINEAR_SOLVER
 pflotran card:: NEWTON_SOLVER
 pflotran card:: LINEAR_SOLVER
 pflotran card:: GRID
 pflotran card:: FLUID_PROPERTY
 "FLUID_PROPERTY,diffusion_coeffient units" set to default value.
 pflotran card:: MATERIAL_PROPERTY
   Name :: Alluvium
 "MATERIAL_PROPERTY,rock density units" set to default value.
 "MATERIAL_PROPERTY,specific heat units" set to default value.
 

In [14]:
%%script bash -s "$pflotran_out_dir"
cd $1
ls *.h5

R1.h5
R10.h5
R11.h5
R12.h5
R13.h5
R14.h5
R15.h5
R16.h5
R17.h5
R18.h5
R19.h5
R2.h5
R20.h5
R21.h5
R22.h5
R23.h5
R24.h5
R25.h5
R26.h5
R27.h5
R28.h5
R29.h5
R3.h5
R30.h5
R4.h5
R5.h5
R6.h5
R7.h5
R8.h5
R9.h5
pflotran.h5


<a id='dart_prepare'></a>
# DART files preparation
In this section, the following procedures are performed:
- generate the template for DART generic variable quantity files (i.e., ```DEFAULT_obs_kind_mod.F90``` and ```obs_def_pflotran_mod.f90```);
- generate the DART input namelists;
- generate DART prior NetCDF data ```prior_R[ENS].nc``` from PFLOTRAN's parameter and outputs;
- convert the observation file to DART observation format;
- check ```model_mod.F90``` based on current setting by using the ```check_model_mod``` provided by DART.

## Generate the templates for DART generic variable quantity files
- Run: ```list2dartqty.py``` to sequentially generate
    - a mapping between PFLOTRAN variales and DART generic quantities in ```obs_def_pflotran_mod.F90```
    - the default DART generic quantity definition file ```DEFAULT_obs_kind_mod.F90```
- Code input arguments:
    - <span style="background-color:yellow">obs_type</span>: filename for ```DEFAULT_obs_kind_mod.F90```
    - <span style="background-color:yellow">def_obs_kind</span>: filename for ```obs_def_pflotran_mod.F90```
    - <span style="background-color:yellow">pflotran_parastate_set</span>: a list of variables required to be assimilated

In [15]:
%%script bash -s "$to_dartqty" "$obs_type" "$def_obs_kind" "$pflotran_parastate_set"
python $1 $2 $3 $4

No new DART variable quantity is added...
Finished generating the ../obs_kind/DEFAULT_obs_kind_mod.f90...
Finished generating the ../obs_type/obs_def_pflotran_mod.f90...


In [16]:
%%script bash -s "$obs_type"
cat $1

! BEGIN DART PREPROCESS KIND LIST
!TEMPERATURE,  QTY_PFLOTRAN_TEMPERATURE, COMMON_CODE
!FLOW_FLUX,  QTY_PFLOTRAN_FLOW_FLUX, COMMON_CODE
!POROSITY,  QTY_PFLOTRAN_POROSITY, COMMON_CODE
!THERMAL_CONDUCTIVITY,  QTY_PFLOTRAN_THERMAL_CONDUCTIVITY, COMMON_CODE
! END DART PREPROCESS KIND LIST


## Generate  DART input namelists in ```input.nml```

The ```input.nml``` file is generated based on a template ```input.nml.template``` by modifying the following namelist entries:

```input.nml.template``` $\rightarrow$ ```input.nml```

|filter_nml|obs_kind_nml|preprocess_nml|model_nml|convertnc_nml|
|:--:|:--:|:--:|:--:|:--:|
| input_state_file_list, output_state_file_list, ens_size, async, adv_ens_command, obs_sequence_in_name | assimilate_these_obs_types | input_files, input_obs_kind_mod_file | time_step_days, time_step_seconds, nvar, var_names, template_file, var_qtynames | netcdf_file, out_file |

**Namelists from DART**
- [filter_nml](https://www.image.ucar.edu/DAReS/DART/Manhattan/assimilation_code/modules/assimilation/filter_mod.html): namelist of the main module for driving ensemble filter assimilations
- [obs_kind_nml](https://www.image.ucar.edu/DAReS/DART/Manhattan/assimilation_code/modules/observations/obs_kind_mod.html#Namelist): namelist for controling what observation types are to be assimilated
- [preprocess_nml](https://www.image.ucar.edu/DAReS/Codes/DART/manhattan/assimilation_code/programs/preprocess/preprocess): namelist of the DART-supplied preprocessor program which creates observation kind and observation definition modules from a set of other specially formatted Fortran 90 files

**Self-defined namelists**
- model_nml: a self-defined namelist for providing the basic information in the model
    - time_step_days, time_step_seconds: the assimilation time window
    - template_file: the template prior NetCDF file for ```model_mod.F90``` to digest the spatial information of the model
    - var_names: the original variable names
    - var_qtynames: the corresponding DART variable quantities
    - nvar: the number of variables
- convertnc_nml: a self-defined namelist for providing the NetCDF observation file name and the DART observation file name used in ```convert_nc.f90```
    - netcdf_file: the location of the NetCDF file containing the observation data
    - out_file: the location of the DART observation file

**Note that**
- There are more namelists or other items in the above namelist in input.nml.template. Users can edit the below python dictionary ```inputnml``` to include their modifications.
- Users can also include more namelists provided by DART by modifying ```inputnml```.

***************
Assemble all the namelists in input.nml

In [17]:
# Parameters for different namelists in input.nml
filter_nml = {"input_state_file_list":dart_input_list,
              "output_state_file_list":dart_output_list,
              "ens_size":nens,
              "async":2,
              "adv_ens_command":advance_model,
              "obs_sequence_in_name":obs_dart}
obs_kind_nml = {"assimilate_these_obs_types":obs_var_set}
model_nml = {"time_step_days":time_step_days,
             "time_step_seconds":time_step_seconds,
             "nvar":len(pflotran_parastate_set),
             "var_names":pflotran_parastate_set,
             "template_file":dart_prior_template,
             "var_qtynames":['QTY_PFLOTRAN_'+v for v in pflotran_parastate_set]}
preprocess_nml = {"input_files":obs_type,
                  "input_obs_kind_mod_file":def_obs_kind}
convertnc_nml = {"netcdf_file": obs_nc,
                 "out_file": obs_dart}
inputnml = {"filter_nml":filter_nml,
            "obs_kind_nml":obs_kind_nml,
            "model_nml":model_nml,
            "preprocess_nml":preprocess_nml,
            "convertnc_nml":convertnc_nml}

# Save it in a temperory pickle file
with open(input_nml_dict, 'wb') as f:
    pickle.dump(inputnml, f)

***************
- Run: ```prepare_inputnml.py```
- Code input arguments:
    - <span style="background-color:yellow">input_nml</span>: the ```input.nml``` namelist file
    - <span style="background-color:yellow">input_nml_dict</span>: the ```inputnml.p``` pickle file

In [18]:
%%script bash -s  "$prep_inputnml" "$input_nml" "$input_nml_dict"
python $1 $2 $3

Finished generating the input namelist file...


## Generate DART prior NetCDF files from the model spin-up
- The structure of ```prior_R[ENS].nc``` file (```[ENS]``` refers to the ensemble number):

| NetCDF dimensions |                      NetCDF variables                      |
|:-----------------:|:----------------------------------------------------------:|
| time: 1           | time: shape(time)                                          |
| x_location: nx    | x_location: shape(x_location)                              |
| y_location: ny    | y_location: shape(y_location)                              |
| z_location: nz    | z_location: shape(z_location)                              |
| member: 1         | member: shape(member)                                      |
|                   | physical variable: shape(x_location,y_location,z_location) |

**Note that** 
- required by DART, each ```prior_R[ENS].nc``` file only includes the state/parameter values of one ensemble member at one given time. 
- For the time, we set the initial time as 0, with time units converted *day* (requied by DART's ```read_model_time``` subroutine). 
- Also, it is different from the definition for the [observation NetCDF](#observationconvertion), because ```prior_R[ENS].nc``` aims for the structured cartesian grids while the observation NetCDF aims for a general case.

**Run the code**
- Run: ```prepare_prior_nc.py``` to generate 
    - the prior input file ```prior_R[ENS].nc``` used by DART
    - the prior template file (copied from ```prior_R1.nc```) used by ```input.nml```
- Code input arguments:
    - <span style="background-color:yellow">pflotran_out</span>: filename ```R[ENS].h5``` from PFLOTRAN model output
    - <span style="background-color:yellow">pflotran_para</span>: pflotran parameter HDF file ```parameter.h5```
    - <span style="background-color:yellow">dart_prior_nc</span>: filename ```prior_R[ENS].nc``` for the prior input file for DART
    - <span style="background-color:yellow">dart_input_list</span>: filename for recording the list of dart_prior_nc
    - <span style="background-color:yellow">nens</span>: number of ensemble
    - <span style="background-color:yellow">spinup</span>: whether it is spinup (if yes, the time is set to zero; otherwise, the time is read from ```R[ENS].h5```)
    - <span style="background-color:yellow">pflotran_parastate_set</span>: a list of variables to be assimilated

In [19]:
%%script bash -s "$prep_prior_nc" "$pflotran_out" "$pflotran_para" "$dart_prior_nc" "$dart_input_list" "$nens" "$spinup" "$pflotran_parastate_set"
python $1 $2 $3 $4 $5 $6 $7 $8

Converting state/parameter into NetCDF file for ensemble 1...
Converting state/parameter into NetCDF file for ensemble 2...
Converting state/parameter into NetCDF file for ensemble 3...
Converting state/parameter into NetCDF file for ensemble 4...
Converting state/parameter into NetCDF file for ensemble 5...
Converting state/parameter into NetCDF file for ensemble 6...
Converting state/parameter into NetCDF file for ensemble 7...
Converting state/parameter into NetCDF file for ensemble 8...
Converting state/parameter into NetCDF file for ensemble 9...
Converting state/parameter into NetCDF file for ensemble 10...
Converting state/parameter into NetCDF file for ensemble 11...
Converting state/parameter into NetCDF file for ensemble 12...
Converting state/parameter into NetCDF file for ensemble 13...
Converting state/parameter into NetCDF file for ensemble 14...
Converting state/parameter into NetCDF file for ensemble 15...
Converting state/parameter into NetCDF file for ensemble 16...
C

In [20]:
%%script bash -s "$dart_prior_template"
ncdump -h $1

netcdf prior_Rtemplate {
dimensions:
	x_location = 1 ;
	y_location = 1 ;
	z_location = 64 ;
	time = 1 ;
	member = 1 ;
variables:
	double time(time) ;
		time:units = "day" ;
		time:calendar = "none" ;
		time:type = "dimension_value" ;
	double member(member) ;
		member:type = "dimension_value" ;
	double x_location(x_location) ;
		x_location:units = "m" ;
		x_location:type = "dimension_value" ;
	double y_location(y_location) ;
		y_location:units = "m" ;
		y_location:type = "dimension_value" ;
	double z_location(z_location) ;
		z_location:units = "m" ;
		z_location:type = "dimension_value" ;
	double TEMPERATURE(z_location, y_location, x_location) ;
		TEMPERATURE:type = "observation_value" ;
		TEMPERATURE:unit = "[C]" ;
	double FLOW_FLUX(z_location, y_location, x_location) ;
		FLOW_FLUX:type = "observation_value" ;
		FLOW_FLUX:unit = "" ;
	double POROSITY(z_location, y_location, x_location) ;
		POROSITY:type = "observation_value" ;
		POROSITY:unit = "" ;
	double THERMAL_CONDUCTIVITY(z_locat

<a id='observationconvertion'></a>
## Convert observation to DART observation format
In this section, the observation data is converted in DART format. We first convert observation data in raw format into NetCDF format. Then, the observation file is converted into DART format. The structure of NetCDF file for recording observation file.

| NetCDF dimensions |           NetCDF variables          |
|:-----------------:|:-----------------------------------:|
| time: 1           | time: shape(time)                   |
| location: nloc    | location: shape(location)           |
|                   | physical variable: shape(time,nloc) |

**Note that** 
- if the time calendar follows *gregorian*, the time unit should be entered as ```seconds since YYYY-MM-DD HH:MM:SS```. Otherwise, put the time calender as *None* and time unit as ```second``` (make sure convert your measurement times to seconds).

***************
- Run: ```csv2nc.py``` to convert the raw csv temperature observations to NetCDF file
- Code input arguments:
    - <span style="background-color:yellow">obs_original</span>: filename for the original observed temperature file
    - <span style="background-color:yellow">obs_nc</span>: filename for the observation NetCDF file

In [21]:
%%script bash -s "$csv_to_nc" "$obs_original" "$obs_nc"
python $1 $2 $3

Finished converting raw observation in NetCDF format...


In [22]:
%%script bash -s "$obs_nc"
ncdump -h $1

netcdf obs_pflotran {
dimensions:
	time = 8641 ;
	location = 5 ;
variables:
	double time(time) ;
		time:units = "seconds since 2017-04-01 00:00:00" ;
		time:calendar = "gregorian" ;
		time:type = "dimension_value" ;
	double x_location(location) ;
		x_location:units = "m" ;
		x_location:type = "dimension_value" ;
	double y_location(location) ;
		y_location:units = "m" ;
		y_location:type = "dimension_value" ;
	double z_location(location) ;
		z_location:units = "m" ;
		z_location:type = "dimension_value" ;
	double TEMPERATURE(location, time) ;
		TEMPERATURE:_FillValue = -99999. ;
		TEMPERATURE:unit = "C" ;
		TEMPERATURE:type = "observation_value" ;
}


***************
- Run: ```prepare_convert_nc.py``` to prepare the ```convert_nc.f90``` based on the list of observation variables.
- Code input arguments:
    - <span style="background-color:yellow">obs_nc</span>: filename for the observation NetCDF file

In [23]:
%%script bash -s "$prep_convert_nc" "$obs_nc"
python $1 $2

In [24]:
%%script bash -s "$utils_dir"
cd $1
head convert_nc.f90

! DART software - Copyright UCAR. This open source software is provided
! by UCAR, "as is", without charge, subject to all terms of use at
! http://www.image.ucar.edu/DAReS/DART/DART_download
!
! Revised from convert_madis_profiler.f90 written by Nancy Colin
! $Id: convert_nc.f90 2019-09-09 15:48:00Z peishi.jiang@pnnl.gov $

program convert_nc

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


***************
- Run shell script: ```dart_seq_convert.csh``` to 
    - preprocess the DART generic variable quantity files prepared by the previous section 
    - generate an executable file for converting observation file in NetCDF format to DART format used by the next section

In [25]:
%%script bash -s "$work_dir" "$compile_convert_nc"
cd $1
csh $2

---------------------------------------------------------------
Removing *.o *.mod files


---------------------------------------------------------------
NetCDF converters build number 1 is preprocess
 Makefile is ready.
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/types_mod.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/utilities_mod.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/time_manager_mod.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/null_mpi_utilities_mod.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/programs/preprocess/preprocess.f90
gfortran time_manager_mod.o types_mo

.........................................

***************
- Run: ```convert_nc``` to convert the observation file in NetCDF to DART format

In [26]:
%%script bash -s "$work_dir" "$convert_nc"
cd $1
$2


 --------------------------------------
 Starting ... at YYYY MM DD HH MM SS = 
                 2019  9 20 14 38 17
 Program convert_nc
 --------------------------------------

  set_nml_output No echo of NML values
  write_obs_seq  opening formatted observation sequence file "../dart_inout/obs_seq_pflotran.out"

 --------------------------------------------------------
 -------------- ASSIMILATE_THESE_OBS_TYPES --------------
    TEMPERATURE
 --------------------------------------------------------
 -------------- EVALUATE_THESE_OBS_TYPES   --------------
    none
 --------------------------------------------------------
 ---------- USE_PRECOMPUTED_FO_OBS_TYPES   --------------
    none
 --------------------------------------------------------

  write_obs_seq  closed observation sequence file "../dart_inout/obs_seq_pflotran.out"

 --------------------------------------
 Finished ... at YYYY MM DD HH MM SS = 
                 2019  9 20 14 38 19
 ------------------------------------

In [27]:
%%script bash -s "$obs_dart"
head -n21 $1

 obs_sequence
obs_kind_definitions
           1
           1 TEMPERATURE                                                     
  num_copies:            1  num_qc:            1
  num_obs:        43205  max_num_obs:        43205
Observation                                                     
Data QC                                                         
  first:            1  last:        43205
 OBS            1
   5.3399999999999999     
   1.0000000000000000     
          -1           2          -1
obdef
loc3Dxyz
     0.000000000000000         0.000000000000000       -0.1000000000000000E-01
kind
           1
     0     152031
   1.0000000000000000     
 OBS            2


## Run ```check_model_mod```
- Run shell script: ```check_model_mod.csh``` to check the model_mod.F90 interface. See details of this mod in [link](https://www.image.ucar.edu/DAReS/DART/Manhattan/assimilation_code/programs/model_mod_check/model_mod_check.html).

In [28]:
%%script bash -s "$work_dir" "$compile_model_check_mod"
cd $1
csh $2



---------------------------------------------------------------
pflotran build number 1 is preprocess
 Makefile is ready.
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/types_mod.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/utilities_mod.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/time_manager_mod.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/programs/preprocess/preprocess.f90
gfortran -O2 -ffree-line-length-none -I/usr/local/Cellar/netcdf/4.6.3_1/include  -c	../../../assimilation_code/modules/utilities/null_mpi_utilities_mod.f90
gfortran types_mod.o time_manager_mod.o utilities_mod.o preprocess.o null_mpi_utilities_mod.o -o preprocess  -O2 -ffree-line-length-no

rm: No match.
...................................................

<a id='run_dart_pflotran'></a>
# (TODO) Run DART and PFLOTRAN
In this section, run the shell script to couple DART and PFLOTRAN

In [ ]:
# %%script bash -s "$work_dir" "$run_filter"
# cd $1
# csh $2